#Kaggle Competition

In this session lab, we are going to compete in a Kaggle Competition.

First, we are going to upload the `train` and `test` datasets to databricks using the following route:

*Data -> Add Data -> Upload File*

**Note:** You have the option to select the location to store the files within DBFS.

Once the files are uploaded, we can use them in our environment.

You will need to change /FileStore/tables/train.csv with the name of the files and the path(s) that you chose to store them.

**Note 1:** When the upload is complete, you will get a confirmation along the path and name assigned. Filenames might be slightly modified by Databricks.

**Note 2:** If you missed the path and filename message you can navigate the DBFS via: *Data -> Add Data -> Upload File -> DBFS* or checking the content of the path `display(dbutils.fs.ls("dbfs:/FileStore/some_path"))`


# Lecture des données et import.

Dans cette partie on se consacre à l'import des packages et à la lecture des tables préalablement importées dans databricks.

In [3]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.ml import Pipeline
import numpy as np
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import RFormula
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import LogisticRegression


In [4]:
train_data = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferSchema='true').load('/FileStore/tables/train_set-51e11.csv')
test_data = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferSchema='true').load('/FileStore/tables/test_set-b5f57.csv')

allcol=train_data.columns
print(train_data.printSchema())
display(train_data)

Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
1,2611,326,20,120,27,1597,168,214,184,2913,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6
2,2772,324,17,42,7,1814,175,220,183,2879,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2764,4,14,480,-21,700,201,212,148,700,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
4,3032,342,9,60,8,4050,202,227,164,2376,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
5,2488,23,11,117,21,1117,209,218,151,1136,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
6,2968,83,8,390,19,4253,232,226,127,4570,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
7,3027,11,6,534,47,1248,214,228,151,2388,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2
8,3216,277,9,67,23,5430,212,236,169,2373,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9,3242,262,5,849,169,1672,207,242,173,691,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
10,3315,61,15,120,-6,3042,231,208,106,1832,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,7


#Normalisation
Selon l'algorythme de machine learning implémenté, la normalisation des données peut grandement améliorer la qualité du modèle comme par exemple dans le cas de régretion logistique. On va donc vérifier si les données à normaliser le sont.   
Tout d'abord il faut savoir qu'on ne normalisera que les colonnes numérique donc les colonnes `wilderness areas` et `soil type` ne doivent pas etre traité .On ne normalisera pas la colonne à prédire.

In [6]:
display(train_data.describe())

summary,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
count,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720,528720
mean,264360.5,2959.8005522771978,155.62989105764865,14.095042744742019,269.73184483280374,46.47471818732032,2352.595141095476,212.15306211227116,223.34397034347103,142.55561544863065,1979.363977152368,0.44879520351036467,0.05155091541836889,0.4362554849447723,0.06339839612649417,0.005178544409139053,0.012984188228173703,0.008303071569072477,0.021251323952186412,0.0027840823119987894,0.011338704796489636,1.929187471629596E-4,3.1585716447268873E-4,0.002055908609471932,0.055927523074595246,0.021385610531093963,0.05153956725677107,0.03004804055076411,0.001040248146466939,1.8913602663035254E-6,0.004911862611590255,0.005870782266606143,0.0033060977454985627,0.006871311847480708,0.01603495233772129,0.0014449992434558934,0.05761272507187169,0.09947420184596761,0.03710281434407626,8.246330761083372E-4,0.004558178241791496,0.0019027084279013467,0.0015452413375699804,0.1982731880768649,0.05159630806476018,0.043858753215312456,0.09036541080344984,0.0776781661370858,0.0027916477530640035,0.003275835981237706,2.0237554849447724E-4,4.7094870630957783E-4,0.02691594794976547,0.023645786049326677,0.01511764260856408,2.0522942200030263
stddev,152628.46150046852,280.35176605588595,112.09547000611657,7.501853213925286,213.0691243000493,58.43393334432487,1563.5576789297904,26.793767287222455,19.813478898016417,38.33617756122474,1325.8775894507655,0.4973716283609164,0.2211187712817278,0.49592045930374445,0.2436783777901742,0.07177560052505284,0.11320610991987054,0.09074219605744721,0.14422116530619197,0.05269095224733794,0.10587804197508767,0.013888192622839862,0.017769580634794634,0.04529553763604428,0.22978192944758896,0.14466618739799658,0.22109575463981399,0.17071968525602896,0.03223612873317405,0.0013752673435748917,0.06991241278744903,0.07639585866250666,0.057403603506716985,0.08260817046651969,0.12560996171829092,0.03798570717667918,0.2330098746199943,0.299297935909551,0.1890139229873284,0.028704609643629447,0.0673602986542089,0.04357856950849074,0.03927921186678901,0.3986994251559165,0.22121080807928184,0.2047809617527026,0.28670482870531167,0.26766472339111985,0.052762294502499626,0.057141150280363213,0.014224449912744647,0.02169626244175406,0.1618379721615066,0.15194310289287732,0.1220210131544082,1.3968634944281586
min,1,1759,-72,-3,-144,-182,-805,-2,0,-17,-704,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
max,528720,3957,440,65,1397,601,7577,268,266,259,7548,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7


Comme le tableau ci dessus le montre, la table n'a pas été normalisée

In [8]:
# Selection des colonnes à normaliser
selection_n= ["Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology", "Horizontal_Distance_To_Roadways", "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm", "Horizontal_Distance_To_Fire_Points"]

# mise en place du pipeline
vector_assembler_n = VectorAssembler(inputCols=selection_n, outputCol="features_n")
normalizer = Normalizer(inputCol="features_n", outputCol="normFeatures", p=2.0)
pipeline = Pipeline(stages=[vector_assembler_n, normalizer])

# Normalisation de la table d'entrainement (calcul)
normalised = pipeline.fit(train_data).transform(train_data)
step_normalize= sqlContext.createDataFrame(
  normalised.select("Id","normFeatures").rdd.map(lambda x: np.hstack(([x[0]],x[1])).tolist()),
  ["IDs"]+selection_n
) 

# Normalisation de la table de test (calcul)
normalised_test = pipeline.fit(test_data).transform(test_data)
step_normalize_test= sqlContext.createDataFrame(
  normalised_test.select("Id","normFeatures").rdd.map(lambda x: np.hstack(([x[0]],x[1])).tolist()),
  ["IDs"]+selection_n
)

#Attribution des nouvelles valeurs
selection_nn=allcol
for col in selection_n:
  try:
    selection_nn.remove(col)
  except:
    print(col)
print(selection_nn)


step_normalize = step_normalize.join(train_data.select(selection_nn), train_data.Id==step_normalize.IDs).drop('IDs')
step_normalize.cache()
selection_nn.remove("Cover_Type")

step_normalize_test = step_normalize_test.join(test_data.select(selection_nn), test_data.Id==step_normalize_test.IDs).drop('IDs')
step_normalize_test.cache()

['Id', 'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40', 'Cover_Type']
 Out[ 29 ]: DataFrame[Elevation: double, Aspect: double, Slope: double, Horizontal_Distance_To_Hydrology: double, Vertical_Distance_To_Hydrology: double, Horizontal_Distance_To_Roadways: double, Hillshade_9am: double, Hillshade_Noon: double, Hillshade_3pm: double, Horizontal_Distance_To_Fire_Points: double, Id: int, Wilderness_Area1: int, Wilderness_Area2: int, Wilderness_Area3: int, Wilderness_Area4: int, Soil_Type1: int, Soil_Type2: int, Soil_Type3: int, Soil_Type4: int, Soil_Type5: int, Soil_Type6: int, Soil_Type7: int, Soil_Type8: int, Soil_Type9: int, Soil_Type10: int, Soil_Type11: int, Soil_Type12: int, Soil_Type13: int, Soil_Type14: int, Soil_Type15: int, Soil_Type16: int, Soil_Type17: int, Soil_Type18: int, Soil_Type19: int, Soil_Type20: int, Soil_Type21: int, Soil_Type22: int, Soil_Type23: int, Soil_Type24: int, Soil_Type25: int, Soil_Type26: int, Soil_Type27: int, Soil_Type28: int, Soil_Type29: int, Soil_Type30: int, Soil_Type31: int, Soil_Type32: int, Soil_Type33: int, Soil_Type34: int, Soil_Type35: int, Soil_Type36: int, Soil_Type37: int, Soil_Type38: int, Soil_Type39: int, Soil_Type40: int]

Vérifions maintenant que les données ont été normalisée

In [10]:
display(step_normalize.describe())

#Feature selection

On s'interesse maintenant à l'interet de chacune de nos variables pour déterminer le `Cover_Type`.  
Pour chaque colonne, on va regarder si elles sont statistiquement significatives pour optimiser notre selection.

In [12]:
#Separation des colonnes pour les test statistiques
selection_s1= ["Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology", 
               "Horizontal_Distance_To_Roadways", "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm", "Horizontal_Distance_To_Fire_Points"]
selection_s2= ['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4']
selection_s3= ['Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9',
 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 'Soil_Type17',
 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']

#Mise en place du pipeline 
vector_assembler_s1 = VectorAssembler(inputCols=selection_s1, outputCol="features_s1")
vector_assembler_s2 = VectorAssembler(inputCols=selection_s2, outputCol="features_s2")
vector_assembler_s3 = VectorAssembler(inputCols=selection_s3, outputCol="features_s3")
selector_s1 = ChiSqSelector(fwe=0.00001, featuresCol="features_s1",
                         outputCol="selectedFeatures_s1", labelCol="Cover_Type")
selector_s2 = ChiSqSelector(fwe=0.00001, featuresCol="features_s2",
                         outputCol="selectedFeatures_s2", labelCol="Cover_Type")
selector_s3 = ChiSqSelector(fwe=0.00001, featuresCol="features_s3",
                         outputCol="selectedFeatures_s3", labelCol="Cover_Type")
pipeline = Pipeline(stages=[vector_assembler_s1, vector_assembler_s2,vector_assembler_s3])

#Execution de celui ci
featured = pipeline.fit(step_normalize).transform(step_normalize)
selected_s1=selector_s1.fit(featured).selectedFeatures
selected_s2=selector_s2.fit(featured).selectedFeatures
selected_s3=selector_s3.fit(featured).selectedFeatures


--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-557558875004595> in <module> () 
 13 vector_assembler_s2 = VectorAssembler ( inputCols = selection_s2 , outputCol = "features_s2" ) 
 14 vector_assembler_s3 = VectorAssembler ( inputCols = selection_s3 , outputCol = "features_s3" ) 
 ---> 15 selector_s1 = ChiSqSelector(fwe=0.00001, featuresCol="features_s1",
 16 outputCol="selectedFeatures_s1", labelCol="Cover_Type")
 17 selector_s2 = ChiSqSelector(fwe=0.00001, featuresCol="features_s2",

 NameError : name 'ChiSqSelector' is not defined

In [13]:
selection_test=[]
selection_test.append("Id")
formula= "Cover_Type ~"
print("Les colonnes sélectionnées sont : ")
for ind in selected_s1:
  print(selection_s1[ind])
  selection_test.append(selection_s1[ind])
  formula +=selection_s1[ind]+"+"
for ind in selected_s2:
  print(selection_s2[ind]) 
  selection_test.append(selection_s2[ind])
  formula +=selection_s1[ind]+"+"
for ind in selected_s3:
  print(selection_s3[ind])
  selection_test.append(selection_s3[ind])
  formula +=selection_s3[ind]+"+"
  
formula = formula[:-1]


#Machine learning / Prédiction

Une fois les données normalisés et la selection effectué, on peut cherche le modèle qui offre la meilleur prédiction.

## Modèle choisi / Decisiontree

In [15]:
# Mise en place du pipeline puis execution
dt = DecisionTreeClassifier(labelCol="Cover_Type", featuresCol="featureml",maxDepth=30)
vector_assembler_final = VectorAssembler(inputCols=selection_test, outputCol="featureml")

pipeline = Pipeline(stages=[vector_assembler_final,dt])
ModelDT = pipeline.fit(step_normalize)
predictions = ModelDT.transform(step_normalize_test)


Il est interessant de regarder les résultats obtenus par notre modèle sur le jeu d'entrainement

In [17]:
display(ModelDT.transform(step_normalize))

Enfin on va prédire les données de test avant soumissions

In [19]:
predictions = ModelDT.transform(step_normalize_test)
predictions = predictions.withColumn("Cover_Type", predictions["prediction"].cast("int"))  
predictions.cache()
display(predictions.select("Id","Cover_Type"))

Id,Cover_Type
528928,2
528931,2
529057,1
529105,1
529136,3
529699,5
529740,2
530045,1
530119,2
530172,7


## Autres modèles
### Regression logistique

In [21]:
lr = LinearRegression(maxIter=10)

paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

evaluator = MulticlassClassificationEvaluator(
    labelCol="Cover_Type", predictionCol="prediction", metricName="accuracy"
)

crossvalLR = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

#cvModelLR = crossvalLR.fit(step_normalize)
#predictions = cvModelLR.transform(step_normalize_test)

### Random forest

In [23]:
rf = RandomForestClassifier(labelCol="Cover_Type", featuresCol="featureml", numTrees=10)

pipeline = Pipeline(stages=[vector_assembler_final,rf])

evaluator = MulticlassClassificationEvaluator(
    labelCol="Cover_Type", predictionCol="prediction", metricName="accuracy"
)
  
paramGrid = ParamGridBuilder() \
    .addGrid(rf.maxDepth,[5,15,25])\
    .addGrid(rf.maxBins,[2,4,6,8]) \
    .build()

crossvalRF = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

#cvModelRF = crossvalTF.fit(step_normalize)
#predictions = cvModelRF.transform(step_normalize_test)



### Multi layer perceptron

In [25]:
layers1= [48, 40, 30, 20, 8]
layers2= [48, 30, 30, 30, 8]


pipeline = Pipeline(stages=[vector_assembler_final,trainer])

evaluator = MulticlassClassificationEvaluator(
    labelCol="Cover_Type", predictionCol="prediction", metricName="accuracy"
)
  
paramGrid = ParamGridBuilder() \
    .addGrid(trainer.layers,[layers1,layers2])\
    .build()

crossvalMLP = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

#cvModelMLP = crossvalMLP.fit(step_normalize)
#predictions = cvModelMLP.transform(step_normalize_test)


# Conclusion et ouverture

Le meilleur modèle sélectioné est l'arbre de décision avec un score de 0.86. 


Beaucoup d'autres algo auraient pu être mis en place, mais les performances de Databricks community ne l'ont pas permis, 
notamment plus de jeux avec les hyper parametres avec la cross validation et ou le traintestSplit.  
Il est interessant de constater l'importance de la puissance des machines sur la création d'un modèle. Le jeu entre puissance de calcul à choix d'algorythme est a prendre en compte dans la phase de modélisation.

# Markdown d'origine

In [28]:
print('Train data size: {} rows, {} columns'.format(train_data.count(), len(train_data.columns)))
print('Test data size: {} rows, {} columns'.format(test_data.count(), len(test_data.columns)))

Train data size: 528720 rows, 56 columns
Test data size: 226595 rows, 55 columns

We will use the `VectorAssembler()` to merge our feature columns into a single vector column as requiered by Spark methods.

In [30]:
print(len(selection_test))
display(step_normalize.describe())

For this example, we will use `Logistic Regression`.

Now, we are going to create a pipeline that will chain the vector assambler and the classifier stages.

Once we have trained the classifier, we can use it to make predictions on the test data.

Finally, we can create a file with the predictions.

In [35]:
# Select columns Id and prediction
(predictions
 .repartition(1)
 .select('Id', 'Cover_Type')
 .write
 .format('com.databricks.spark.csv')
 .options(header='true')
 .mode('overwrite')
 .save('/FileStore/kaggle-submission'))

To be able to download the predictions file, we need its name (`part-*.csv`):

In [37]:
display(dbutils.fs.ls("dbfs:/FileStore/kaggle-submission/part-00000-tid-7884458236033938986-fd76e6e1-e98c-4a28-a51f-870368fa46b3-1156-c000.csv"))

Files stored in /FileStore are accessible in your web browser via `https://<databricks-instance-name>.cloud.databricks.com/files/`.
  
For this example:

https://community.cloud.databricks.com/files/kaggle-submission/part-*.csv?o=######

where `part-*.csv` should be replaced by the name displayed in your system  and the number after `o=` is the same as in your Community Edition URL.


Finally, we can upload the predictions to kaggle and check what is the perfromance.